In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from config2 import omdb_key, tmdb_key, pwrd
import warnings
from pprint import pprint
import re
import time
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')

In [ ]:
#create url list for API calls to The Movie Database for the first 500 pages of movies
url_list = []


for i in range(1,501):
    num = i
    url = "https://api.themoviedb.org/3/discover/movie?api_key="+tmdb_key+"&language=en-US&&sort_by=vote_count.desc&include_adult=false&include_video=false&page=" + str(num)
    url_list.append(url)


In [ ]:
len(url_list)

In [ ]:
#loop through url's and make API call for each

movies_list = []

for i in url_list:
    r = requests.get(i)
    print(r)
    r = r.json()
    movies_list.append(r["results"])
    

In [ ]:
len(movies_list)

In [ ]:
#get movies from each page into a list (there are 20 moives per page)
m_list = []

for i in range(len(movies_list)):
    for j in range(20):
        m_list.append(movies_list[i][j].items())

In [ ]:
print(f"There are {len(m_list)} movies in the database")

In [ ]:
#put movies in data frame
mymovies_df = pd.DataFrame(m_list)
mymovies_df.head()

In [ ]:
#rename columns
mymovies_df.columns = ["popularity", "tmdb_vote_count", "video", "poster_path", "id", "adult", "backdrop_path", "original_language", "original_title", "genre_ids", "title", "vote_average", "overview", "release_date"]

In [ ]:
print(mymovies_df.columns)

In [ ]:
#create smaller df with the columns we want
short_df = mymovies_df[['original_title','overview']].copy()
short_df.head()

In [ ]:
#remove tuple from each cell and only show data
for x in short_df.columns:
    short_df[x] = [y[1] for y in short_df[x]]


In [ ]:
short_df.head()

In [ ]:
#create list for json results and url variables for API call
movie_jsons = []
url1 = "http://www.omdbapi.com/?t="
url2 = "&apikey="

In [ ]:
#loop through movies and make API call for each title
for movie in short_df["original_title"]:
    url = url1 + movie + url2 + omdb_key
    response = requests.get(url)
    print(response)
    response = response.json()
    movie_jsons.append(response)
    time.sleep(.2)

In [ ]:
 #create dataframe
df1 = pd.DataFrame(movie_jsons)

In [ ]:
#set max column view and check dataframe
pd.set_option('display.max_columns', 999)
df1.head()

In [ ]:
#check data count
df1.count()

In [ ]:
#drop rows that have no ratings
df1 = df1[pd.notnull(df1['Ratings'])]

In [ ]:
#drop rows that have no Box Office
df1 = df1[pd.notnull(df1['BoxOffice'])]

In [ ]:
df1.count()

In [ ]:
total_ratings = [len(data) for data in df1["Ratings"]]

In [ ]:
total_ratings.count(3)

In [ ]:
#remove movies that dont have ratings from all 3 sources
df1 = df1[df1["Ratings"].map(len)==3]

In [ ]:
df1.count()

In [ ]:
#create list of all ratings in the dataframe
ratings_list = []
for data in df1["Ratings"]:
    for i in data:
        ratings_list.append(i)
ratings_list[:5]

In [ ]:
#create spearate list for each source
imdb = []
r_t = []
metacritic = []
for d in ratings_list:
    if d["Source"] == "Internet Movie Database":
        imdb.append(d["Value"])
    elif d["Source"] == "Rotten Tomatoes":
        r_t.append(d["Value"])
    else:
        metacritic.append(d["Value"])
        

In [ ]:
#check sample list
print(len(imdb))
print(len(r_t))
print(len(metacritic))

In [ ]:
#convert strings to floats and ints
imdb = [float(x.split("/")[0]) for x in imdb]
r_t = [int(x.rstrip("%")) for x in r_t]
metacritic = [int(x.split("/")[0]) for x in metacritic]

In [ ]:
#check results
print(imdb[:5])
print(r_t[:5])
print(metacritic[:5])

In [ ]:
print(len(imdb))
print(len(r_t))
print(len(metacritic))

In [ ]:
#create new columns
df1["IMDB"] = imdb
df1["Rotten Tomatoes"] = r_t
df1["Metacritic"] = metacritic

In [ ]:
#check dataframe 
df1

In [ ]:
#drop unwanted columns
df1 = df1.drop(["Country", "DVD", "Error", "Language", "Metascore", "Ratings", "Response", "imdbRating", "Website"], axis = 1)

In [ ]:
#create variable for regex test
string = df1["Awards"][0]

In [ ]:
#test regex code
more_wins = re.search(r'(\d+) win', string)
if more_wins:
    print(more_wins.group(1))
else:
    print("No more wins")
more_noms = re.search(r'(\d+) nominations', string)
if more_noms:
    print(more_noms.group(1))
    print(type(more_noms.group(1)))
    print(int(more_noms.group(1)))
else:
    print("No more noms")
big_noms = re.search(r'Nominated for (\d+)', string)
if big_noms:
    print(big_noms.group(1))
    print(type(big_noms.group(1)))
else:
    print("No big noms")
big_wins = re.search(r'Won (\d+)', string)
if big_wins:
    print(big_wins.group(1))
else:
    print("No big wins")

In [ ]:
#create wins and noms list. Prase through 

wins_list = []
noms_list = []

for x in df1["Awards"]:
    wins = 0
    noms = 0
    big_wins = re.search(r'Won (\d+)', x)
    if big_wins:
        wins += int(big_wins.group(1))
    big_noms = re.search(r'Nominated for (\d+)', x)
    if big_noms:
        noms += int(big_noms.group(1))
    more_wins = re.search(r'(\d+) win', x)
    if more_wins:
        wins += int(more_wins.group(1))
    more_noms = re.search(r'(\d+) nominations', x)
    if more_noms:
        noms += int(more_noms.group(1))
    wins_list.append(wins)
    noms_list.append(noms)
    
print(wins_list[:5])
print(noms_list[:5])
    

In [ ]:
#adds wins and nominations to df1
df1["Wins"] = wins_list
df1["Nominations"] = noms_list

In [ ]:
#convert runtime to int
df1["Runtime"] = [int(x.split(" ")[0]) for x in df1["Runtime"]]


In [ ]:
#drop rows that has no imdbVotes
df1 = df1[~df1["imdbVotes"].str.contains("N/A")]


In [ ]:
df1.count()

In [ ]:
#convert imdbVotes to string
df1["imdbVotes"] = [int(x.replace(',','')) for x in df1["imdbVotes"]]


In [ ]:
#check dataframe
df1.head()

In [ ]:
#save as csv
df1.to_csv("data/movies.csv")

In [11]:
#reload csv for SQL Load
df1 = pd.read_csv("data/movies.csv") 

In [12]:
print(df1.columns)

Index(['Unnamed: 0', 'Actors', 'Awards', 'BoxOffice', 'Director', 'Genre',
       'Plot', 'Poster', 'Production', 'Rated', 'Released', 'Runtime', 'Title',
       'Type', 'Writer', 'Year', 'imdbID', 'imdbVotes', 'totalSeasons', 'IMDB',
       'Rotten Tomatoes', 'Metacritic', 'Wins', 'Nominations'],
      dtype='object')


In [3]:
#drop Unnamed column
df1 = df1.drop(['Unnamed: 0'], axis=1)

In [13]:
#rename Rotten Tomatoes column
df1.rename(columns = {'Rotten Tomatoes':'Rotten_Tomatoes'}, inplace = True) 

In [14]:
#connect to local database
rds_connection_string = "postgres:"+ pwrd + "@localhost:5432/movie_finder"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [15]:
#export movie data to SQL
df1.to_sql(name='movies', con=engine, if_exists='replace', index=True)

In [16]:
#check tables
engine.table_names()

['movies']

In [17]:
#add primary key
engine.execute('alter table movies add primary key(index)')

In [18]:
#read tables from SQL
pd.read_sql("SELECT * FROM movies",con=engine)


,index,Unnamed: 0,Actors,Awards,BoxOffice,Director,Genre,Plot,Poster,Production,...,Writer,Year,imdbID,imdbVotes,totalSeasons,IMDB,Rotten_Tomatoes,Metacritic,Wins,Nominations
0,0,0,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...",Won 4 Oscars. Another 152 wins & 209 nominations.,"$292,568,851",Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",A thief who steals corporate secrets through t...,https://m.media-amazon.com/images/M/MV5BMjAxMz...,Warner Bros. Pictures,...,Christopher Nolan,2010,tt1375666,1902038,None,8.8,87,74,156,209
1,1,1,"Ryan Reynolds, Karan Soni, Ed Skrein, Michael ...",Nominated for 2 Golden Globes. Another 27 wins...,"$328,674,489",Tim Miller,"Action, Adventure, Comedy, Sci-Fi",A wisecracking mercenary gets experimented on ...,https://m.media-amazon.com/images/M/MV5BYzE5Mj...,20th Century Fox,...,"Rhett Reese, Paul Wernick",2016,tt1431045,848182,None,8.0,85,65,27,75
2,2,2,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",Nominated for 1 Oscar. Another 38 wins & 79 no...,"$623,279,547",Joss Whedon,"Action, Adventure, Sci-Fi",Earth's mightiest heroes must come together an...,https://m.media-amazon.com/images/M/MV5BNDYxNj...,Walt Disney Pictures,...,"Joss Whedon (screenplay), Zak Penn (story), Jo...",2012,tt0848228,1208826,None,8.0,91,69,38,80
3,3,3,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Won 2 Oscars. Another 152 wins & 155 nominations.,"$533,316,061",Christopher Nolan,"Action, Crime, Drama, Thriller",When the menace known as the Joker wreaks havo...,https://m.media-amazon.com/images/M/MV5BMTMxNT...,Warner Bros. Pictures/Legendary,...,"Jonathan Nolan (screenplay), Christopher Nolan...",2008,tt0468569,2141768,None,9.0,94,84,154,155
4,4,4,"Ellen Burstyn, Matthew McConaughey, Mackenzie ...",Won 1 Oscar. Another 43 wins & 143 nominations.,"$158,737,441",Christopher Nolan,"Adventure, Drama, Sci-Fi",A team of explorers travel through a wormhole ...,https://m.media-amazon.com/images/M/MV5BZjdkOT...,Paramount Pictures,...,"Jonathan Nolan, Christopher Nolan",2014,tt0816692,1353310,None,8.6,72,74,44,143
5,5,5,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",Won 3 Oscars. Another 85 wins & 128 nominations.,"$749,700,000",James Cameron,"Action, Adventure, Fantasy, Sci-Fi",A paraplegic Marine dispatched to the moon Pan...,https://m.media-amazon.com/images/M/MV5BMTYwOT...,20th Century Fox,...,James Cameron,2009,tt0499549,1069221,None,7.8,82,83,88,128
6,6,6,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin D...",Nominated for 2 Oscars. Another 52 wins & 99 n...,"$270,592,504",James Gunn,"Action, Adventure, Comedy, Sci-Fi",A group of intergalactic criminals must pull t...,https://m.media-amazon.com/images/M/MV5BMTAwMj...,Walt Disney Pictures,...,"James Gunn, Nicole Perlman, Dan Abnett (based ...",2014,tt2015381,985657,None,8.0,91,76,52,101
7,7,7,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier",Nominated for 1 Oscar. Another 10 wins & 34 no...,None,David Fincher,Drama,An insomniac office worker and a devil-may-car...,https://m.media-amazon.com/images/M/MV5BMmEzNT...,20th Century Fox,...,"Chuck Palahniuk (novel), Jim Uhls (screenplay)",1999,tt0137523,1729708,None,8.8,79,66,10,35
8,8,8,"Robert Downey Jr., Terrence Howard, Jeff Bridg...",Nominated for 2 Oscars. Another 20 wins & 65 n...,"$318,298,180",Jon Favreau,"Action, Adventure, Sci-Fi","After being held captive in an Afghan cave, bi...",https://m.media-amazon.com/images/M/MV5BMTczNT...,Paramount Pictures,...,"Mark Fergus (screenplay), Hawk Ostby (screenpl...",2008,tt0371746,889014,None,7.9,94,79,20,67
9,9,9,"Jamie Foxx, Christoph Waltz, Leonardo DiCaprio...",Won 2 Oscars. Another 56 wins & 151 nominations.,"$162,804,648",Quentin Tarantino,"Drama, Western","With the help of a German bounty hunter, a fre...",https://m.media-amazon.com/images/M/MV5BMjIyNT...,The Weinstein Co.,...,Quentin Tarantino,2012,tt1853728,1253112,None,8.4,86,81,58,151
